In [2]:
import tensorflow as tf

2022-04-23 18:05:46.916458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 18:05:46.924632: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
print(tf.__version__)

2.7.0


In [84]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1,1800))
])


In [85]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 1800)              0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [10]:
x = np.random.random((2, 3))

In [11]:
type(x)

numpy.ndarray

# Conectando com servidor

In [12]:
import requests
import pandas as pd


In [13]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemParaTeste"

In [16]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [61]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [97]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')

In [98]:
X = np.stack(dfTratado.values)

In [99]:
X.shape

(3, 1800)

In [100]:
Y = np.stack(df['campoY'].values)

In [101]:
Y = Y.reshape(3,1)

In [102]:
Y.shape

(3, 1)

# Minha Rede

In [107]:
# Build the Model
model_tune = keras.models.Sequential([
  keras.layers.SimpleRNN(1800, return_sequences=True, input_shape=(1,1800)),
  keras.layers.SimpleRNN(1800),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 100.0)
])

model_tune.build()


In [108]:
model_tune.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_15 (SimpleRNN)   (None, 1, 1800)           6481800   
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 1800)              6481800   
                                                                 
 dense_7 (Dense)             (None, 1)                 1801      
                                                                 
 lambda_7 (Lambda)           (None, 1)                 0         
                                                                 
Total params: 12,965,401
Trainable params: 12,965,401
Non-trainable params: 0
_________________________________________________________________


In [110]:
model_tune.fit(X, Y, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

  

In [109]:
model_tune.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])